In [ ]:
import sys
from hashlib import sha512
from pathlib import Path

import h5py as h5
import numpy as np
import pandas as pd

sys.path.append('../lib')
from storage import get_storage_functions
from local_paths import preproc_dir, eye_latency_calib_paths

In [ ]:
#============================================================================
# session
#============================================================================
sess_name = 'sess_name'

#============================================================================
# main
#============================================================================
fr_change_frac_thres = .5

#============================================================================
# paths
#============================================================================
proc_dir = preproc_dir
output_dir = preproc_dir
overwrite = False  # overwrite any existing results; use with caution

# Check parameters and whether already done

In [ ]:
preproc_main_path = Path(proc_dir) / (sess_name + '-main.nwb')
rasters_path = Path(proc_dir) / (sess_name + '-rasters.nwb')
print('Loading session from', preproc_main_path)
print('Loading rasters from', rasters_path)
preproc_main_path = preproc_main_path.expanduser()
rasters_path = rasters_path.expanduser()
assert preproc_main_path.is_file()
assert rasters_path.is_file()

print('Using eye tracker latency calibration in:\n\t'+'\n\t'.join(map(str,eye_latency_calib_paths)))
eye_latency_calib_paths = tuple(Path(v).expanduser() for v in eye_latency_calib_paths)
assert all(v.is_file() for v in eye_latency_calib_paths)

output_dir = Path(output_dir)
assert output_dir.expanduser().is_dir()
output_path = output_dir / (sess_name + '-proc.h5')
print('Saving results to', output_path)
output_path = output_path.expanduser()

In [ ]:
if not overwrite and output_path.is_file():
    with h5.File(output_path, 'r') as f:
        try:
            if f['progress_report/shared_processing/all_done'][()].item():
                raise RuntimeError(f'{sess_name} has already been processed')
        except KeyError:
            pass

In [ ]:
save_results, add_attr_to_dset, check_equals_saved, link_dsets, copy_group = \
    get_storage_functions(output_path, overwrite=overwrite)

# Main

### Save a default random seed (per-session, deterministic)

In [ ]:
random_seed = int(sha512(bytes(sess_name, 'utf-8')).hexdigest()[-8:], 16)
print('random_seed:', random_seed)
save_results('config/default_random_seed', random_seed)

### Save image size

In [ ]:
with h5.File(preproc_main_path, 'r') as f:
    im_size_px = f['stimulus/templates/stimuli/dimension'][()]
    stim_desc = f['general/stimulus'][()].decode()

print('stimuli:', stim_desc)
ppd = stim_desc.split('pix per degree')[0].split('[')[-1].strip('] ').split(',')
ppd = np.array(list(map(float, ppd)))
im_size_dva = np.round(im_size_px / ppd, 1)

print('image size (pixels):', im_size_px)
print('image size (dva):', im_size_dva)

In [ ]:
save_results('stimulus/size_px', im_size_px)
save_results('stimulus/size_dva', im_size_dva)
save_results('stimulus/pixels_per_degree', ppd)

### Save stimulus folder(s): root and any subdirs

In [ ]:
with h5.File(preproc_main_path, 'r') as f:
    uim_paths = f['stimulus/templates/stimuli/external_file'][()].astype(str)

# find longest shared root dir
if any('\\' in v for v in uim_paths):  # Windows paths
    uim_subds = np.char.split(uim_paths, '\\')
else:
    uim_subds = np.char.split(uim_paths, '/')
for i in range(min(map(len, uim_subds))):
    if len(set(v[i] for v in uim_subds)) > 1:
        i -= 1
        break

im_root = '/'.join(uim_subds[0][:i+1])
uim_fns = np.array([v[-1] for v in uim_subds])
uim_subds = np.array(['/'.join(v[i+1:-1]) for v in uim_subds])

In [ ]:
save_results('stimulus/folder', im_root)
save_results('stimulus/subdirectories', np.array(sorted(set(uim_subds))).astype(bytes))

### Get (rig-specific, measured) eye latency

In [ ]:
tracker, rig = pd.read_csv(eye_latency_calib_paths[0], converters={'Eye tracker': str})\
    .set_index('Name').loc[sess_name][['Eye tracker', 'Rig']]
tracker = tracker.split(' ')[-1]
tracker_latency = pd.read_csv(eye_latency_calib_paths[1], converters={'Tracker': str})\
    .set_index(['Rig', 'Tracker']).loc[(rig, tracker)]['Latency']
tracker_latency_rule = f'rig {rig}, tracker: {tracker}'
print(f'Correcting eye tracking timing by {tracker_latency} ms ({tracker_latency_rule})')

In [ ]:
save_results('config/eye_tracking_latency', tracker_latency, attrs=dict(unit='ms'))
save_results('config/eye_tracking_latency_rule', tracker_latency_rule)

### Save formatted fixations

In [ ]:
fix_df = {'Time': 'start_time', 'End time': 'stop_time', 'Trial': 'trial_id', 'Screen X': 'x', 'Screen Y': 'y'}
with h5.File(preproc_main_path, 'r') as f:
    try:
        group = f['processing/behavior/fixations']
    except KeyError:  # compat
        group = f['processing/fixation_detection/fixations']
    for k, v in fix_df.items():
        fix_df[k] = group[v][()]
        if k == 'Trial':
            fix_df[k] = fix_df[k].astype(int)  # pytables does not like uint64 as an index

fix_df = pd.DataFrame(fix_df)

Compensate for tracker latency before further formatting

In [ ]:
fix_df['Time'] -= tracker_latency/1e3
fix_df['End time'] -= tracker_latency/1e3

Cross-reference fixations to stimulus presentations

In [ ]:
with h5.File(preproc_main_path, 'r') as f:
    pres_iim = f['stimulus/presentation/presentations/data'][()]
    pres_tid = f['intervals/presentations/trial_id'][()]
    pres_t0s = f['intervals/presentations/start_time'][()]
    pres_t1s = f['intervals/presentations/stop_time'][()]
    pres_pos = f['intervals/presentations/position'][()]

im_fns = uim_fns[pres_iim]
im_subdirs = uim_subds[pres_iim]
pres_df = pd.DataFrame({
    'Trial': pres_tid,
    'Start time': pres_t0s,
    'End time': pres_t1s,
    'X': pres_pos[:,0],
    'Y': pres_pos[:,1],
    'Image filename': im_fns,
    'Image subdir': im_subdirs,
}).set_index('Trial')

In [ ]:
# not all fixations were during image presentation; select those that overlap in part with pres time
fix_df = fix_df[fix_df['Trial'].isin(pres_tid)]
pres_df_ = pres_df.loc[fix_df['Trial']]
in_pres = (
    ((fix_df['End time'].values - pres_df_['Start time'].values) > 0)
    & ((fix_df['Time'].values - pres_df_['End time'].values) <= 0)
)
fix_df = fix_df[in_pres]
pres_df_ = pres_df.loc[fix_df['Trial']]

# find and clip fixations across stim onset to onset time
is_zeroth = fix_df['Time'].values <= pres_df_['Start time'].values
fix_df.loc[is_zeroth, 'Time'] = pres_df_.loc[is_zeroth, 'Start time'].values

# cast presentation-specific info to fixations
fix_df[['Relative X', 'Relative Y']] = fix_df[['Screen X', 'Screen Y']] - pres_df_[['X', 'Y']].values
fix_df[['Image filename', 'Image subdir']] = pres_df_[['Image filename', 'Image subdir']].values

# add order of fixation in each trial
fix_df['Fixation'] = pd.concat([
    pd.Series(data=np.argsort(g['Time'])+1, index=g.index)
    for _, g in fix_df.groupby('Trial')
])
# order starts with 0 only for trials containing a "zeroth fixation"; 1 otherwise
m = fix_df['Trial'].isin(fix_df[is_zeroth]['Trial'])
fix_df.loc[m, 'Fixation'] -= 1

# add duration, time-since-stim-on; convert time s -> ms
fix_df['Duration'] = fix_df['End time'] - fix_df['Time']
fix_df['Trial time'] = fix_df['Time'] - pres_df_['Start time'].values
assert (fix_df['Trial time'][fix_df['Fixation'] == 0] == 0).all()  # sanity checks
assert (fix_df['Trial time'][fix_df['Fixation'] != 0] > 0).all()
fix_df['Trial time'] = np.clip(fix_df['Trial time'], 0, None)
fix_df[['Time', 'Trial time', 'End time', 'Duration']] *= 1e3

# drop irrelevant columns; set index
fix_df = fix_df.drop(columns=['Screen X', 'Screen Y'])
fix_df = fix_df.set_index(['Trial', 'Fixation'])

# add ref to prev fixation
ipre = np.full(len(fix_df), -1, dtype=int)
for i, (itr, ifix) in enumerate(fix_df.index):
    try:
        ipre[i] = fix_df.index.get_loc((itr, ifix-1))
    except KeyError:
        pass
fix_df['Preceding fixation index'] = ipre

print(fix_df.shape)
fix_df.head()

In [ ]:
fix_df.to_hdf(output_path, 'fixation_dataframe', mode='a', format='table', complevel=9, complib='zlib')

### Select neurons

Using minimal criteria to only exclude clearly technical problems:
1. FR in 2nd half of data must not be 0 (if so, it is most likely the unit was lost)
2. FR in 2nd half of data must not be > 150% or < 50% of overall FR (if so, it is most likely the unit was not stable)

Criteria more meaningful for downstream analysis (e.g., visual responsiveness) can be applied downstream.

In [ ]:
with h5.File(rasters_path, 'r') as f:
    unit_names = f['processing/ecephys/unit_names/unit_name'][()].astype(str)
    rasters = f['processing/ecephys/rasters/data'][()]
duration = len(rasters)
all_unit_names = unit_names.copy()

l = rasters.shape[0]
fr = rasters.mean(0)*1e3
fr1 = rasters[l//2:].mean(0)*1e3

dfr = fr1 / fr - 1

m0 = fr1 == 0
m1 = np.abs(dfr) > fr_change_frac_thres
m = m1|m0

print('total', m.size, 'units:')
print(f'excluding {m0.sum()} ({m0.mean()*100:.1f}%) unit(s) without spikes in second half of recording')
print(f'excluding {m1.sum()} ({m1.mean()*100:.1f}%) unstable unit(s)')
print(f'excluding {m.sum()} ({m.mean()*100:.1f}%) total unit(s)')
unit_names = unit_names[~m]
print('keeping', (~m).sum(), 'units:')
print('\t' + '\n\t'.join(str(unit_names).split('\n')))

In [ ]:
save_results('recording_duration', duration, attrs=dict(unit='ms'))
save_results('config/unit_selection/fr_change_frac_thres', fr_change_frac_thres)
save_results('unit_selection/all', all_unit_names.astype(bytes))
save_results('unit_selection/simple', unit_names.astype(bytes))

### Calculate response stats
I.e., response mean and stdev, during stimulus and inter-stimulus periods respectively

In [ ]:
is_stim = np.zeros(len(rasters), dtype=bool)
for t0, t1 in np.round(np.array([pres_t0s, pres_t1s]).T).astype(int):
    is_stim[t0:t1] = True
byper_mean_std = np.empty((2, 2, rasters.shape[1]))
for iper in range(2):
    m = is_stim if iper else ~is_stim
    vals = rasters[m]
    byper_mean_std[iper, 0] = vals.mean(0) * 1e3
    byper_mean_std[iper, 1] = vals.std(0) * 1e3

In [ ]:
save_results(
    'response_stats/stim_spont_mean_std',
    byper_mean_std,
    attrs=dict(
        dims=np.array(['period', 'statistic', 'unit'], dtype=bytes),
        period=np.array(['inter-stimulus', 'stimulus'], dtype=bytes),
        statistic=np.array(['mean', 'stdev'], dtype=bytes),
        unit=all_unit_names.astype(bytes),
        num_ms=np.array([(~is_stim).sum(), is_stim.sum()]),
        unit_='spikes/s',
    )
)

# Wrap up

In [ ]:
save_results('progress_report/shared_processing/all_done', True)

In [ ]:
%load_ext watermark
%watermark -vm --iversions -rbg

# Quick plots

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8,3))

for i, ax in enumerate(axs):
    vals = byper_mean_std[:,i]

    ax.scatter(*vals, s=10)
    u, l = vals.min(), vals.max()
    ax.set_aspect('equal')
    ax.plot([u,l], [u,l], lw=1, ls='--', zorder=-1, color='gray')
    if i == 0:
        ax.set_title('Mean')
    else:
        ax.set_title('Stdev.')
    ax.set_xlabel('Inter-stimulus period')
    ax.set_ylabel('Stimulus period')